# Setup

In [22]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
from collections import defaultdict
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.cider.cider import Cider
from typing import List, Tuple
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


# Defining The Architectures

In [23]:
class BahdanauAttention(nn.Module):
    def __init__(self, encoder_dim, decoder_dim, attention_dim):
        super(BahdanauAttention, self).__init__()
        self.encoder_att = nn.Linear(encoder_dim, attention_dim)
        self.decoder_att = nn.Linear(decoder_dim, attention_dim)
        self.full_att = nn.Linear(attention_dim, 1)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, encoder_out, decoder_hidden):
        att1 = self.encoder_att(encoder_out)
        att2 = self.decoder_att(decoder_hidden)
        att = self.relu(att1 + att2.unsqueeze(1))
        att = self.full_att(att)
        alpha = self.softmax(att.squeeze(2))
        context = (encoder_out * alpha.unsqueeze(2)).sum(dim=1)
        return context, alpha

class CNNEncoder(nn.Module):
    def __init__(self, encoded_image_size=7):
        super(CNNEncoder, self).__init__()
        self.enc_image_size = encoded_image_size

        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )

        self.block4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )

        self.block5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )

    def forward(self, images):
        x = self.block1(images)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)

        batch_size = x.size(0)
        x = x.permute(0, 2, 3, 1)
        x = x.view(batch_size, -1, 512)
        return x

class LSTMDecoder(nn.Module):
    def __init__(self, attention_dim, embed_dim, decoder_dim, vocab_size,
                 encoder_dim=512, dropout=0.5):
        super(LSTMDecoder, self).__init__()

        self.encoder_dim = encoder_dim
        self.attention_dim = attention_dim
        self.embed_dim = embed_dim
        self.decoder_dim = decoder_dim
        self.vocab_size = vocab_size
        self.dropout = dropout

        self.attention = BahdanauAttention(encoder_dim, decoder_dim, attention_dim)
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.dropout_layer = nn.Dropout(p=self.dropout)
        self.lstm_cell = nn.LSTMCell(embed_dim + encoder_dim, decoder_dim)
        self.init_h = nn.Linear(encoder_dim, decoder_dim)
        self.init_c = nn.Linear(encoder_dim, decoder_dim)
        self.fc = nn.Linear(decoder_dim, vocab_size)

    def init_hidden_state(self, encoder_out):
        mean_encoder_out = encoder_out.mean(dim=1)
        h = self.init_h(mean_encoder_out)
        c = self.init_c(mean_encoder_out)
        return h, c

class ImageCaptioningModel(nn.Module):
    def __init__(self, vocab_size, attention_dim=512, embed_dim=256,
                 decoder_dim=512, encoder_dim=512, dropout=0.5):
        super(ImageCaptioningModel, self).__init__()

        self.encoder = CNNEncoder()
        self.decoder = LSTMDecoder(
            attention_dim=attention_dim,
            embed_dim=embed_dim,
            decoder_dim=decoder_dim,
            vocab_size=vocab_size,
            encoder_dim=encoder_dim,
            dropout=dropout
        )

# Loading The Model

In [24]:
def load_model(checkpoint_path='best_model.pth'):
    print(f"Loading checkpoint from {checkpoint_path}...")

    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

    vocab = checkpoint['vocab']

    model = ImageCaptioningModel(
        vocab_size=len(vocab),
        attention_dim=512,
        embed_dim=256,
        decoder_dim=512,
        encoder_dim=512,
        dropout=0.5
    ).to(device)

    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    print(f"Model loaded successfully")
    print(f"Trained for {checkpoint['epoch']} epochs")
    print(f"Best validation loss: {checkpoint['val_loss']:.4f}")
    print(f"Vocabulary size: {len(vocab)}")

    return model, vocab

# Loading Images

In [25]:
def load_and_preprocess_image(image_path: str) -> torch.Tensor:
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)

    return image

# Caption Generation

In [26]:
def generate_caption(image_path: str, model: ImageCaptioningModel,
                    vocab, max_length: int = 20) -> str:
    image = load_and_preprocess_image(image_path).to(device)

    with torch.no_grad():
        encoder_out = model.encoder(image)

    h, c = model.decoder.init_hidden_state(encoder_out)

    current_word = torch.tensor([vocab.word2idx['<START>']]).to(device)

    generated_words = []

    with torch.no_grad():
        for _ in range(max_length):
            word_emb = model.decoder.embedding(current_word)

            context, alpha = model.decoder.attention(encoder_out, h)

            lstm_input = torch.cat([word_emb, context], dim=1)
            h, c = model.decoder.lstm_cell(lstm_input, (h, c))

            logits = model.decoder.fc(h)  # (1, vocab_size)
            predicted_word_idx = torch.argmax(logits, dim=1).item()

            predicted_word = vocab.idx2word[predicted_word_idx]
            if predicted_word == '<END>':
                break

            if predicted_word not in ['<START>', '<PAD>', '<UNK>']:
                generated_words.append(predicted_word)

            current_word = torch.tensor([predicted_word_idx]).to(device)

    caption = ' '.join(generated_words)

    return caption

# Generating Captions With Beam Search

In [27]:
def generate_caption_beam_search(image_path: str, model: ImageCaptioningModel,
                                vocab, beam_width: int = 3, max_length: int = 20) -> str:
    image = load_and_preprocess_image(image_path).to(device)

    with torch.no_grad():
        encoder_out = model.encoder(image)
        encoder_out = encoder_out.expand(beam_width, -1, -1)

    h, c = model.decoder.init_hidden_state(encoder_out)

    start_word = vocab.word2idx['<START>']
    beams = [([start_word], 0.0, h[0:1], c[0:1])]

    completed_beams = []

    with torch.no_grad():
        for step in range(max_length):
            all_candidates = []

            for seq, score, h_beam, c_beam in beams:
                last_word = seq[-1]

                if vocab.idx2word[last_word] == '<END>':
                    completed_beams.append((seq, score))
                    continue

                word_tensor = torch.tensor([last_word]).to(device)
                word_emb = model.decoder.embedding(word_tensor)

                context, alpha = model.decoder.attention(encoder_out[0:1], h_beam)

                lstm_input = torch.cat([word_emb, context], dim=1)
                h_new, c_new = model.decoder.lstm_cell(lstm_input, (h_beam, c_beam))

                logits = model.decoder.fc(h_new)
                log_probs = torch.log_softmax(logits, dim=1)

                top_log_probs, top_indices = torch.topk(log_probs, beam_width)

                for i in range(beam_width):
                    next_word = top_indices[0, i].item()
                    next_log_prob = top_log_probs[0, i].item()

                    new_seq = seq + [next_word]
                    new_score = score + next_log_prob

                    all_candidates.append((new_seq, new_score, h_new, c_new))

            beams = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_width]

            if len(beams) == 0:
                break

        completed_beams.extend([(seq, score) for seq, score, _, _ in beams])

    if completed_beams:
        best_seq, _ = max(completed_beams, key=lambda x: x[1])
    else:
        best_seq = beams[0][0] if beams else [start_word]

    caption_words = [vocab.idx2word[idx] for idx in best_seq
                     if vocab.idx2word[idx] not in ['<START>', '<END>', '<PAD>']]

    return ' '.join(caption_words)

# Visualization

In [28]:
def visualize_prediction(image_path: str, model: ImageCaptioningModel,
                        vocab, reference_captions: List[str] = None):
    generated_caption = generate_caption(image_path, model, vocab)

    image = Image.open(image_path).convert('RGB')

    plt.figure(figsize=(12, 8))

    plt.imshow(image)
    plt.axis('off')

    title = f"Generated: {generated_caption}\n"
    if reference_captions:
        title += "\nReference captions:\n"
        for i, ref in enumerate(reference_captions[:3], 1):  # Show max 3 references
            title += f"{i}. {ref}\n"

    plt.title(title, fontsize=12, pad=20)
    plt.tight_layout()
    plt.show()

# Evaluating On Test Set

In [29]:
def evaluate_on_test_set(model: ImageCaptioningModel, test_loader, vocab):
    model.eval()

    references = defaultdict(list)
    hypotheses = {}

    print("Generating captions for test set...")

    image_id = 0
    with torch.no_grad():
        for images, captions_batch, lengths in test_loader:
            images = images.to(device)

            for i in range(images.size(0)):
                image = images[i:i+1]

                encoder_out = model.encoder(image)

                h, c = model.decoder.init_hidden_state(encoder_out)
                current_word = torch.tensor([vocab.word2idx['<START>']]).to(device)
                generated_words = []

                for _ in range(20):
                    word_emb = model.decoder.embedding(current_word)
                    context, alpha = model.decoder.attention(encoder_out, h)
                    lstm_input = torch.cat([word_emb, context], dim=1)
                    h, c = model.decoder.lstm_cell(lstm_input, (h, c))
                    logits = model.decoder.fc(h)
                    predicted_word_idx = torch.argmax(logits, dim=1).item()

                    predicted_word = vocab.idx2word[predicted_word_idx]
                    if predicted_word == '<END>':
                        break

                    if predicted_word not in ['<START>', '<PAD>', '<UNK>']:
                        generated_words.append(predicted_word)

                    current_word = torch.tensor([predicted_word_idx]).to(device)

                hypotheses[image_id] = [' '.join(generated_words)]

                caption_indices = captions_batch[i]
                ref_words = [vocab.idx2word[idx.item()] for idx in caption_indices
                            if vocab.idx2word[idx.item()] not in ['<START>', '<END>', '<PAD>']]
                references[image_id] = [' '.join(ref_words)]

                image_id += 1

    print(f"Generated captions for {image_id} test images")

    print()
    print("Computing BLEU scores...")
    bleu_scorer = Bleu(n=4)
    bleu_score, _ = bleu_scorer.compute_score(references, hypotheses)

    print("Computing CIDEr score...")
    cider_scorer = Cider()
    cider_score, _ = cider_scorer.compute_score(references, hypotheses)

    scores = {
        'BLEU-1': bleu_score[0],
        'BLEU-2': bleu_score[1],
        'BLEU-3': bleu_score[2],
        'BLEU-4': bleu_score[3],
        'CIDEr': cider_score
    }

    return scores

# Analyzing Predictions

In [30]:
def analyze_predictions(model: ImageCaptioningModel, test_loader, vocab,
                       num_samples: int = 50):
    model.eval()

    success_cases = []
    failure_cases = []

    print("Analyzing predictions...")
    print()

    sample_count = 0
    with torch.no_grad():
        for images, captions_batch, lengths in test_loader:
            if sample_count >= num_samples:
                break

            images = images.to(device)

            for i in range(min(images.size(0), num_samples - sample_count)):
                image = images[i:i+1]

                encoder_out = model.encoder(image)
                h, c = model.decoder.init_hidden_state(encoder_out)
                current_word = torch.tensor([vocab.word2idx['<START>']]).to(device)
                generated_words = []

                for _ in range(20):
                    word_emb = model.decoder.embedding(current_word)
                    context, alpha = model.decoder.attention(encoder_out, h)
                    lstm_input = torch.cat([word_emb, context], dim=1)
                    h, c = model.decoder.lstm_cell(lstm_input, (h, c))
                    logits = model.decoder.fc(h)
                    predicted_word_idx = torch.argmax(logits, dim=1).item()

                    predicted_word = vocab.idx2word[predicted_word_idx]
                    if predicted_word == '<END>':
                        break

                    if predicted_word not in ['<START>', '<PAD>', '<UNK>']:
                        generated_words.append(predicted_word)

                    current_word = torch.tensor([predicted_word_idx]).to(device)

                generated_caption = ' '.join(generated_words)

                caption_indices = captions_batch[i]
                ref_words = [vocab.idx2word[idx.item()] for idx in caption_indices
                            if vocab.idx2word[idx.item()] not in ['<START>', '<END>', '<PAD>']]
                reference_caption = ' '.join(ref_words)

                print()
                print(f"Sample {sample_count + 1}:")
                print(f"Generated: {generated_caption}")
                print(f"Reference: {reference_caption}")

                sample_count += 1

    print()
    print(f"Analyzed {sample_count} predictions")


# Main Inference

In [31]:
if __name__ == "__main__":
    model, vocab = load_model('best_model.pth')

    print("Example 1: Single Image Caption Generation")

    test_image_path = './caption_data/Images/1000268201_693b08cb0e.jpg'

    if os.path.exists(test_image_path):
        caption = generate_caption(test_image_path, model, vocab)
        print(f"Image: {test_image_path}")
        print(f"Generated caption: {caption}")

        visualize_prediction(test_image_path, model, vocab)
    else:
        print(f"Image not found: {test_image_path}")
        print("Please update the path to a valid test image.")

    # Example 2: Batch caption generation
    print()
    print("Example 2: Batch Caption Generation")

    image_dir = './caption_data/Images'
    if os.path.exists(image_dir):
        test_images = [os.path.join(image_dir, f) for f in os.listdir(image_dir)[:10]]

        print(f"\nGenerating captions for {len(test_images)} images...")
        for img_path in test_images:
            caption = generate_caption(img_path, model, vocab)
            print(f"{os.path.basename(img_path)}: {caption}")

    # Example 3: Compare greedy vs beam search
    print()
    print("Example 3: Greedy vs Beam Search")

    if os.path.exists(test_image_path):
        greedy_caption = generate_caption(test_image_path, model, vocab)
        beam_caption = generate_caption_beam_search(test_image_path, model, vocab, beam_width=3)

        print()
        print(f"Image: {os.path.basename(test_image_path)}")
        print(f"Greedy decoding: {greedy_caption}")
        print(f"Beam search (K=3): {beam_caption}")


    print()
    print("Inference demonstrations complete!")


Loading checkpoint from best_model.pth...


FileNotFoundError: [Errno 2] No such file or directory: 'best_model.pth'